In [1]:
#import required libraries
import pandas as pd

import os

import csv

from datetime import datetime

import sklearn
from sklearn.cluster import KMeans

import altair as alt
import numpy as np
import pandas as pd
from sklearn import set_config
from sklearn.model_selection import GridSearchCV, cross_validate, train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import Ridge

#ignore warnings
import warnings
warnings.filterwarnings('ignore')

#display all features
pd.set_option('display.max_columns', None)

In [7]:
data = pd.DataFrame()
df = pd.DataFrame() 
current_directory = os.getcwd()
data['time'] = range(30600, 54000)
for h in range(24,26):
    for i in range(3, 8):
        for j in range(1, 32):
            try:
                relative_path = "dailydata/Sec_%d-%d-%d.csv" % (i, j, h)
                csv_file = os.path.join(current_directory, relative_path)
                new_data = pd.read_csv(csv_file)
                df = pd.concat([df, new_data], ignore_index=True)
            except:
                pass
df 

,year,num,time,month,price,BBBasis,BBUpper,BBLower,MFI,ATR,Mom,day,TTM,color,WaveASlow,WaveAFast,WaveBSlow,WaveBFast,WaveCMCAD,WaveCFast,TICK,TICKBBUpper,TICKBBLower,TICKMA,ADD,PCC,TRIN,TRIN_lastclose,VIX,VIXBBUpper,VIXBBLower,VOLD,change_5_ago,change_15_ago,change_30_ago,change_60_ago,change_120_ago,BBdiff,BBUpdiff,BBLowdiff,MFIchange_5,MFIchange_15,MFIchange_30,MFIchange_60,ATRchange_5,ATRchange_15,ATRchange_30,ATRchange_60,Momchange_5,Momchange_15,Momchange_30,Momchange_60,TTMchange_5,TTMchange_15,TTMchange_30,WaveASlowchange_5,WaveASlowchange_15,WaveASlowchange_30,WaveASlowchange_60,WaveAFastchange_5,WaveAFastchange_15,WaveAFastchange_30,WaveAFastchange_60,WaveBSlowchange_5,WaveBSlowchange_15,WaveBSlowchange_30,WaveBSlowchange_60,WaveBFastchange_5,WaveBFastchange_15,WaveBFastchange_30,WaveBFastchange_60,WaveCMCADchange_5,WaveCMCADchange_15,WaveCMCADchange_30,WaveCMCADchange_60,WaveCFastchange_5,WaveCFastchange_15,WaveCFastchange_30,WaveCFastchange_60,VIXchange_5,VIXchange_15,VIXchange_30,VIXchange_60,VIXchange_120,VIXBBdiff,VIXBBUpdiff,VIXBBLowdiff,TRINchange_5,TRINchange_15,TRINchange_30,TRINchange_60,TICKchange_5,TICKchange_15,TICKchange_30,TICKchange_60,TICKBBUpdiff,TICKBBLowdiff,TICKMADiff,TICKMAchange_5,TICKMAchange_15,TICKMAchange_30,TICKMAchange_60,PCCchange_5,PCCchange_15,PCCchange_30,PCCchange_60,VOLDchange_5,VOLDchange_15,VOLDchange_30,VOLDchange_60,change_5
0,NaN,NaN,30600.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,30601.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,30602.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,30603.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,30604.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [11]:
#removing NA values that exist in change_120 columns
df = df.iloc[0:3600]
df

,year,num,time,month,price,BBBasis,BBUpper,BBLower,MFI,ATR,Mom,day,TTM,color,WaveASlow,WaveAFast,WaveBSlow,WaveBFast,WaveCMCAD,WaveCFast,TICK,TICKBBUpper,TICKBBLower,TICKMA,ADD,PCC,TRIN,TRIN_lastclose,VIX,VIXBBUpper,VIXBBLower,VOLD,change_5_ago,change_15_ago,change_30_ago,change_60_ago,change_120_ago,BBdiff,BBUpdiff,BBLowdiff,MFIchange_5,MFIchange_15,MFIchange_30,MFIchange_60,ATRchange_5,ATRchange_15,ATRchange_30,ATRchange_60,Momchange_5,Momchange_15,Momchange_30,Momchange_60,TTMchange_5,TTMchange_15,TTMchange_30,WaveASlowchange_5,WaveASlowchange_15,WaveASlowchange_30,WaveASlowchange_60,WaveAFastchange_5,WaveAFastchange_15,WaveAFastchange_30,WaveAFastchange_60,WaveBSlowchange_5,WaveBSlowchange_15,WaveBSlowchange_30,WaveBSlowchange_60,WaveBFastchange_5,WaveBFastchange_15,WaveBFastchange_30,WaveBFastchange_60,WaveCMCADchange_5,WaveCMCADchange_15,WaveCMCADchange_30,WaveCMCADchange_60,WaveCFastchange_5,WaveCFastchange_15,WaveCFastchange_30,WaveCFastchange_60,VIXchange_5,VIXchange_15,VIXchange_30,VIXchange_60,VIXchange_120,VIXBBdiff,VIXBBUpdiff,VIXBBLowdiff,TRINchange_5,TRINchange_15,TRINchange_30,TRINchange_60,TICKchange_5,TICKchange_15,TICKchange_30,TICKchange_60,TICKBBUpdiff,TICKBBLowdiff,TICKMADiff,TICKMAchange_5,TICKMAchange_15,TICKMAchange_30,TICKMAchange_60,PCCchange_5,PCCchange_15,PCCchange_30,PCCchange_60,VOLDchange_5,VOLDchange_15,VOLDchange_30,VOLDchange_60,change_5
0,24.0,12.0,37800.0,3.0,516.19,515.03,517.63,512.44,98.78,0.5253,0.73,2.0,2.1700,0.0,1.4600,0.4753,3.32,2.6600,3.96,3.20,74.0,796.395,-478.595,103.111,319.0,1.066,0.83,0.7,14.13,14.64,13.86,28291194.0,-0.14,-0.03,0.13,1.11,2.66,5.19,-1.44,-3.75,-0.36,-0.67,-1.03,27.08,-0.0235,-0.0712,-0.1048,-0.1327,-0.08,-0.77,-1.69,-2.75,-0.3400,-0.7900,-0.9000,-0.1700,-0.3600,-0.5700,-0.4700,-0.1528,-0.3500,-0.6047,-0.7647,-0.12,-0.16,-0.11,0.65,-0.1600,-0.2900,-0.3900,0.0400,-0.04,0.07,0.36,1.59,-0.08,-0.04,0.13,1.15,0.04,0.06,0.02,-0.03,-0.27,0.78,-0.51,-0.27,0.04,-0.20,-0.16,0.00,337.0,-381.0,143.0,-235.0,-722.395,-552.595,-29.111,9.000,-144.556,-203.000,-64.445,-0.007,0.005,-0.027,-0.044,-422775.0,-9371364.0,-3424149.0,14358235.0,NaN
1,24.0,12.0,37801.0,3.0,516.16,515.03,517.63,512.44,98.78,0.5253,0.73,2.0,2.1700,0.0,1.4600,0.4753,3.32,2.6600,3.96,3.20,68.0,796.395,-478.595,102.444,319.0,1.066,0.83,0.7,14.13,14.64,13.86,28298149.0,-0.16,-0.02,0.13,1.05,2.75,5.19,-1.47,-3.72,-0.36,-0.67,-1.03,27.08,-0.0235,-0.0712,-0.1048,-0.1327,-0.08,-0.77,-1.69,-2.75,-0.3400,-0.7900,-0.9000,-0.1700,-0.3600,-0.5700,-0.4700,-0.1528,-0.3500,-0.6047,-0.7647,-0.12,-0.16,-0.11,0.65,-0.1600,-0.2900,-0.3900,0.0400,-0.04,0.07,0.36,1.59,-0.08,-0.04,0.13,1.15,0.05,0.06,0.05,-0.03,-0.27,0.78,-0.51,-0.27,0.04,-0.20,-0.16,0.00,331.0,-393.0,23.0,-237.0,-728.395,-546.595,-34.444,8.333,-145.889,-224.223,-64.667,-0.007,0.005,-0.027,-0.044,-560110.0,-9368788.0,-4099400.0,14365190.0,NaN
2,24.0,12.0,37802.0,3.0,516.15,515.03,517.63,512.44,98.78,0.5253,0.73,2.0,2.1700,0.0,1.4600,0.4753,3.32,2.6600,3.96,3.20,36.0,796.395,-478.595,98.889,318.0,1.066,0.83,0.7,14.13,14.64,13.86,28274154.0,-0.16,-0.03,0.07,1.02,2.79,5.19,-1.48,-3.71,-0.36,-0.67,-1.03,27.08,-0.0235,-0.0712,-0.1048,-0.1327,-0.08,-0.77,-1.69,-2.75,-0.3400,-0.7900,-0.9000,-0.1700,-0.3600,-0.5700,-0.4700,-0.1528,-0.3500,-0.6047,-0.7647,-0.12,-0.16,-0.11,0.65,-0.1600,-0.2900,-0.3900,0.0400,-0.04,0.07,0.36,1.59,-0.08,-0.04,0.13,1.15,0.05,0.06,0.05,-0.03,-0.27,0.78,-0.51,-0.27,0.04,-0.20,-0.16,0.00,301.0,-417.0,-17.0,-231.0,-760.395,-514.595,-62.889,5.000,-148.555,-228.667,-64.000,-0.004,0.005,-0.030,-0.041,-586933.0,-9608963.0,-4651231.0,14426572.0,NaN
3,24.0,12.0,37803.0,3.0,516.16,515.03,517.63,512.44,98.78,0.5253,0.73,2.0,2.1700,0.0,1.4600,0.4753,3.32,2.6600,3.96,3.20,10.0,796.395,-478.595,96.000,313.0,1.064,0.90,0.7,14.13,14.64,13.86,28056379.0,-0.15,-0.07,0.11,1.10,2.71,5.19,-1.47,-3.72,-0.36,-0.67,-1.03,27.08,-0.0235,-0.0712,-0.1048,-0.1327,-0.08,-0.77,-1.69,-2.75,-0.3400,-0.7900,-0.9000,-0.1700,-0.3600,-0.5700,-0.4700,-0.1528,-0.3500,-0.6047,-

In [10]:
#reset index and check if any values are missing from data
df = df.reset_index(drop=True)
mask = (df['time'] == df['time'].shift(-1)) & (df['price'] == df['price'].shift(-1))
df = df[~mask]
df = df.reset_index(drop=True)
df.isnull().any()

year              True
num               True
time             False
month             True
price             True
                 ...  
VOLDchange_5      True
VOLDchange_15     True
VOLDchange_30     True
VOLDchange_60     True
change_5          True
Length: 111, dtype: bool

In [25]:
df.isna().any().any()

False

In [27]:
df['TRIN'].fillna(method='ffill', inplace=True)
df['TRIN_lastclose'].fillna(method='ffill', inplace=True)
df['price'].fillna(method='ffill', inplace=True)
df['BBBasis'].fillna(method='ffill', inplace=True)
df['BBUpper'].fillna(method='ffill', inplace=True)
df['BBLower'].fillna(method='ffill', inplace=True)
df['MFI'].fillna(method='ffill', inplace=True)
df['ATR'].fillna(method='ffill', inplace=True)
df['TTM'].fillna(method='ffill', inplace=True)
df['color'].fillna(method='ffill', inplace=True)
df['WaveASlow'].fillna(method='ffill', inplace=True)
df['WaveAFast'].fillna(method='ffill', inplace=True)
df['WaveBSlow'].fillna(method='ffill', inplace=True)
df['WaveBFast'].fillna(method='ffill', inplace=True)
df['WaveCMCAD'].fillna(method='ffill', inplace=True)
df['WaveCFast'].fillna(method='ffill', inplace=True)
df['Mom'].fillna(method='ffill', inplace=True)
df['ADD'].fillna(method='ffill', inplace=True)
df['TICK'].fillna(method='ffill', inplace=True)
df['TICKBBUpper'].fillna(method='ffill', inplace=True)
df['TICKBBLower'].fillna(method='ffill', inplace=True)
df['TICKMA'].fillna(method='ffill', inplace=True)
df['VOLD'].fillna(method='ffill', inplace=True)
df['VIXBBUpper'].fillna(method='ffill', inplace=True)
df['VIXBBLower'].fillna(method='ffill', inplace=True)
df['VIX'].fillna(method='ffill', inplace=True)
df['PCC'].fillna(method='ffill', inplace=True)
df['TICK'].fillna(method='bfill', inplace=True)
df['TICKBBUpper'].fillna(method='bfill', inplace=True)
df['TICKBBLower'].fillna(method='bfill', inplace=True)
df['TICKMA'].fillna(method='bfill', inplace=True)
df['VOLD'].fillna(method='bfill', inplace=True)
df['PCC'].fillna(method='bfill', inplace=True)
df['ADD'].fillna(method='bfill', inplace=True)
df['VIXBBUpper'].fillna(method='bfill', inplace=True)
df['VIXBBLower'].fillna(method='bfill', inplace=True)
df['VIX'].fillna(method='bfill', inplace=True)
df['TRIN'].fillna(method='bfill', inplace=True)

,year,num,time,month,price,BBBasis,BBUpper,BBLower,MFI,ATR,Mom,day,TTM,color,WaveASlow,WaveAFast,WaveBSlow,WaveBFast,WaveCMCAD,WaveCFast,TICK,TICKBBUpper,TICKBBLower,TICKMA,ADD,PCC,TRIN,TRIN_lastclose,VIX,VIXBBUpper,VIXBBLower,VOLD,change_5_ago,change_15_ago,change_30_ago,change_60_ago,change_120_ago,BBdiff,BBUpdiff,BBLowdiff,MFIchange_5,MFIchange_15,MFIchange_30,MFIchange_60,ATRchange_5,ATRchange_15,ATRchange_30,ATRchange_60,Momchange_5,Momchange_15,Momchange_30,Momchange_60,TTMchange_5,TTMchange_15,TTMchange_30,WaveASlowchange_5,WaveASlowchange_15,WaveASlowchange_30,WaveASlowchange_60,WaveAFastchange_5,WaveAFastchange_15,WaveAFastchange_30,WaveAFastchange_60,WaveBSlowchange_5,WaveBSlowchange_15,WaveBSlowchange_30,WaveBSlowchange_60,WaveBFastchange_5,WaveBFastchange_15,WaveBFastchange_30,WaveBFastchange_60,WaveCMCADchange_5,WaveCMCADchange_15,WaveCMCADchange_30,WaveCMCADchange_60,WaveCFastchange_5,WaveCFastchange_15,WaveCFastchange_30,WaveCFastchange_60,VIXchange_5,VIXchange_15,VIXchange_30,VIXchange_60,VIXchange_120,VIXBBdiff,VIXBBUpdiff,VIXBBLowdiff,TRINchange_5,TRINchange_15,TRINchange_30,TRINchange_60,TICKchange_5,TICKchange_15,TICKchange_30,TICKchange_60,TICKBBUpdiff,TICKBBLowdiff,TICKMADiff,TICKMAchange_5,TICKMAchange_15,TICKMAchange_30,TICKMAchange_60,PCCchange_5,PCCchange_15,PCCchange_30,PCCchange_60,VOLDchange_5,VOLDchange_15,VOLDchange_30,VOLDchange_60,change_5
0,24.0,12.0,37800.0,3.0,516.19,515.03,517.63,512.44,98.78,0.5253,0.73,2.0,2.1700,0.0,1.46,0.4753,3.3200,2.6600,3.96,3.20,74.0,796.395,-478.595,103.111,319.0,1.066,0.83,0.70,14.13,14.64,13.86,28291194.0,NaN,NaN,NaN,NaN,NaN,5.19,-1.44,-3.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.78,-0.51,-0.27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-722.395,-552.595,-29.111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,24.0,12.0,37801.0,3.0,516.16,515.03,517.63,512.44,98.78,0.5253,0.73,2.0,2.1700,0.0,1.46,0.4753,3.3200,2.6600,3.96,3.20,68.0,796.395,-478.595,102.444,319.0,1.066,0.83,0.70,14.13,14.64,13.86,28298149.0,NaN,NaN,NaN,NaN,NaN,5.19,-1.47,-3.72,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.78,-0.51,-0.27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-728.395,-546.595,-34.444,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,24.0,12.0,37802.0,3.0,516.15,515.03,517.63,512.44,98.78,0.5253,0.73,2.0,2.1700,0.0,1.46,0.4753,3.3200,2.6600,3.96,3.20,36.0,796.395,-478.595,98.889,318.0,1.066,0.83,0.70,14.13,14.64,13.86,28274154.0,NaN,NaN,NaN,NaN,NaN,5.19,-1.48,-3.71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.78,-0.51,-0.27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-760.395,-514.595,-62.889,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,24.0,12.0,37803.0,3.0,516.16,515.03,517.63,512.44,98.78,0.5253,0.73,2.0,2.1700,0.0,1.46,0.4753,3.3200,2.6600,3.96,3.20,10.0,796.395,-478.595,96.000,313.0,1.064,0.90,0.70,14.13,14.64,13.86,28056379.0,NaN,NaN,NaN,NaN,NaN,5.19,-1.47,-3.72,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.78,-0.51,-0.27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-786.395,-488.595,-86.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,24.0,12.0,37804.0,3.0,516.17,515.03,517.63,512.44,98.78,0.5253,0.73,2.0,2.1700,0.0,1.46,0.4753,3.3200,2.6600,3.96,3.20,10.0,796.395,-478.595,96.000,310.0,1.064,0.90,0.70,14.13,14.64,13.86,27966151.0,NaN,NaN,NaN,NaN,NaN,5.19,-1.46,-3.73,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.78,-0.51,-0.27,NaN,NaN,

In [30]:
df_5min = df[df['time'] % 300 == 0]
df_5min

,year,num,time,month,price,BBBasis,BBUpper,BBLower,MFI,ATR,Mom,day,TTM,color,WaveASlow,WaveAFast,WaveBSlow,WaveBFast,WaveCMCAD,WaveCFast,TICK,TICKBBUpper,TICKBBLower,TICKMA,ADD,PCC,TRIN,TRIN_lastclose,VIX,VIXBBUpper,VIXBBLower,VOLD,change_5_ago,change_15_ago,change_30_ago,change_60_ago,change_120_ago,BBdiff,BBUpdiff,BBLowdiff,MFIchange_5,MFIchange_15,MFIchange_30,MFIchange_60,ATRchange_5,ATRchange_15,ATRchange_30,ATRchange_60,Momchange_5,Momchange_15,Momchange_30,Momchange_60,TTMchange_5,TTMchange_15,TTMchange_30,WaveASlowchange_5,WaveASlowchange_15,WaveASlowchange_30,WaveASlowchange_60,WaveAFastchange_5,WaveAFastchange_15,WaveAFastchange_30,WaveAFastchange_60,WaveBSlowchange_5,WaveBSlowchange_15,WaveBSlowchange_30,WaveBSlowchange_60,WaveBFastchange_5,WaveBFastchange_15,WaveBFastchange_30,WaveBFastchange_60,WaveCMCADchange_5,WaveCMCADchange_15,WaveCMCADchange_30,WaveCMCADchange_60,WaveCFastchange_5,WaveCFastchange_15,WaveCFastchange_30,WaveCFastchange_60,VIXchange_5,VIXchange_15,VIXchange_30,VIXchange_60,VIXchange_120,VIXBBdiff,VIXBBUpdiff,VIXBBLowdiff,TRINchange_5,TRINchange_15,TRINchange_30,TRINchange_60,TICKchange_5,TICKchange_15,TICKchange_30,TICKchange_60,TICKBBUpdiff,TICKBBLowdiff,TICKMADiff,TICKMAchange_5,TICKMAchange_15,TICKMAchange_30,TICKMAchange_60,PCCchange_5,PCCchange_15,PCCchange_30,PCCchange_60,VOLDchange_5,VOLDchange_15,VOLDchange_30,VOLDchange_60,change_5
0,24.0,12.0,37800.0,3.0,516.19,515.03,517.63,512.44,98.78,0.5253,0.73,2.0,2.1700,0.0,1.4600,0.4753,3.3200,2.6600,3.96,3.20,74.0,796.395,-478.595,103.111,319.0,1.066,0.83,0.70,14.13,14.64,13.86,28291194.0,-0.14,-0.03,0.13,1.11,2.66,5.19,-1.44,-3.75,-0.36,-0.67,-1.03,27.08,-0.0235,-0.0712,-0.1048,-0.1327,-0.08,-0.77,-1.69,-2.75,-0.3400,-0.7900,-0.9000,-0.1700,-0.3600,-0.5700,-0.4700,-0.1528,-0.3500,-0.6047,-0.7647,-0.1200,-0.1600,-0.1100,0.6500,-0.1600,-0.2900,-0.3900,0.0400,-0.04,0.07,0.36,1.59,-0.08,-0.04,0.13,1.15,0.04,0.06,0.02,-0.03,-0.27,0.78,-0.51,-0.27,0.04,-0.20,-0.16,0.00,337.0,-381.0,143.0,-235.0,-722.395,-552.595,-29.111,9.000,-144.556,-203.000,-64.445,-0.007,0.005,-0.027,-0.044,-422775.0,-9371364.0,-3424149.0,14358235.0,NaN
300,24.0,12.0,38100.0,3.0,516.12,515.24,517.47,513.01,98.25,0.5092,0.36,2.0,1.9400,0.0,1.3500,0.3750,3.2700,2.5700,3.98,3.18,-146.0,749.972,-522.972,76.667,196.0,1.065,0.81,0.70,14.13,14.48,13.94,24615880.0,-0.07,-0.19,0.61,0.58,3.30,4.46,-1.35,-3.11,-0.53,-1.11,-1.37,-0.51,-0.0161,-0.0626,-0.1116,-0.1368,-0.37,-0.89,-1.63,-3.78,-0.2300,-0.8200,-0.9800,-0.1100,-0.3600,-0.5100,-0.6700,-0.1003,-0.3388,-0.5237,-0.8950,-0.0500,-0.1800,-0.0600,0.4100,-0.0900,-0.3000,-0.3300,-0.1900,0.02,0.05,0.41,1.34,-0.02,-0.07,0.17,0.89,0.00,0.05,0.00,-0.06,-0.36,0.54,-0.35,-0.19,-0.02,-0.10,-0.17,-0.11,-220.0,-122.0,181.0,-726.0,-895.972,-376.972,-222.667,-26.444,-95.555,-100.000,-159.444,-0.001,0.008,-0.041,-0.032,-3675314.0,-16826714.0,-4814330.0,8535458.0,NaN
600,24.0,12.0,38400.0,3.0,516.16,515.38,517.48,513.28,97.61,0.4975,0.06,2.0,1.5700,0.0,1.1400,0.2000,3.1000,2.3700,3.87,3.04,180.0,754.442,-496.442,13.667,325.0,1.062,0.84,0.70,14.12,14.39,13.98,32619174.0,0.04,-0.17,0.14,0.69,4.65,4.20,-1.32,-2.88,-0.64,-1.53,-2.01,23.70,-0.0117,-0.0513,-0.1233,-0.1260,-0.30,-0.75,-1.93,-4.13,-0.3700,-0.9400,-1.5500,-0.2100,-0.4900,-0.8000,-0.9800,-0.1750,-0.4281,-0.7436,-1.1200,-0.1700,-0.3400,-0.4200,0.0100,-0.2000,-0.4500,-0.6600,-0.5600,-0.11,-0.13,0.02,0.93,-0.14,-0.24,-0.20,0.48,-0.01,0.03,0.06,-0.06,-0.56,0.41,-0.27,-0.14,0.03,0.05,-0.11,-0.09,326.0,443.0,-113.0,14.0,-574.442,-676.442,166.333,-63.000,-80.444,-230.111,-136.222,-0.003,-0.011,-0.010,-0.024,8003294.0,3905205.0,-2314432.0,10849682.0,NaN
900,24.0,12.0,38700.0,3.0,515.65,515.51,517.33,513.68,95.22,0.5077,-0.70,2.0,1.1700,0.0,0.9161,0.0154,2.8900,2.1400,3.73,2.87,-509.0,806.012,-601.812,-50.556,166.0,1.059,0.97,0.70,14.17,14.36,13.99,21294872.0,-0.51,-0.54,-0.57,-0.14,4.24,3.65,-1.68,-1.97,-2.39,-3.56,-4.23,27.64,0.0102,-0.0176,-0.0888,-0.1512,-0.76,-1.43,-2.20,-4.46,-0.4000,-1.0000,-1.79

In [31]:
df_5min['year'].fillna(method='ffill', inplace=True)
df_5min['num'].fillna(method='ffill', inplace=True)
df_5min['month'].fillna(method='ffill', inplace=True)
df_5min['day'].fillna(method='ffill', inplace=True)

In [33]:
df_5min.isna().any()

year             False
num              False
time             False
month            False
price            False
                 ...  
VOLDchange_5     False
VOLDchange_15    False
VOLDchange_30    False
VOLDchange_60    False
change_5          True
Length: 111, dtype: bool

In [34]:
df = pd.concat([df, df2], ignore_index=True)

In [35]:
df['price_5'] = df['price'].shift(-1)
df['price_15'] = df['price'].shift(-3)
df['price_30'] = df['price'].shift(-6)
df['price_60'] = df['price'].shift(-12)
df['price_120'] = df['price'].shift(-24)

df['change_5'] = df['price_5'] - df['price']
df['change_15'] = df['price_15'] - df['price']
df['change_30'] = df['price_30'] - df['price']
df['change_60'] = df['price_60'] - df['price']
df['change_120'] = df['price_120'] - df['price']

df.drop(columns='price_5', inplace=True)
df.drop(columns='price_15', inplace=True)
df.drop(columns='price_30', inplace=True)
df.drop(columns='price_60', inplace=True)
df.drop(columns='price_120', inplace=True)

df = df.iloc[:-24]
df

,year,num,time,price,BBBasis,BBUpper,BBLower,MFI,ATR,TTM,color,Mom,WaveASlow,WaveAFast,WaveBSlow,WaveBFast,WaveCMCAD,WaveCFast,day,month,TICKBBUpper,TICKBBLower,TICKMA,TICK,ADD,PCC,TRIN,TRIN_lastclose,VIX,VIXBBUpper,VIXBBLower,VOLD,change_5_ago,change_15_ago,change_30_ago,change_60_ago,change_120_ago,BBdiff,BBUpdiff,BBLowdiff,MFIchange_5,MFIchange_15,MFIchange_30,MFIchange_60,ATRchange_5,ATRchange_15,ATRchange_30,ATRchange_60,Momchange_5,Momchange_15,Momchange_30,Momchange_60,TTMchange_5,TTMchange_15,TTMchange_30,WaveASlowchange_5,WaveASlowchange_15,WaveASlowchange_30,WaveASlowchange_60,WaveAFastchange_5,WaveAFastchange_15,WaveAFastchange_30,WaveAFastchange_60,WaveBSlowchange_5,WaveBSlowchange_15,WaveBSlowchange_30,WaveBSlowchange_60,WaveBFastchange_5,WaveBFastchange_15,WaveBFastchange_30,WaveBFastchange_60,WaveCMCADchange_5,WaveCMCADchange_15,WaveCMCADchange_30,WaveCMCADchange_60,WaveCFastchange_5,WaveCFastchange_15,WaveCFastchange_30,WaveCFastchange_60,VIXchange_5,VIXchange_15,VIXchange_30,VIXchange_60,VIXchange_120,VIXBBdiff,VIXBBUpdiff,VIXBBLowdiff,TRINchange_5,TRINchange_15,TRINchange_30,TRINchange_60,TICKchange_5,TICKchange_15,TICKchange_30,TICKchange_60,TICKBBUpdiff,TICKBBLowdiff,TICKMADiff,TICKMAchange_5,TICKMAchange_15,TICKMAchange_30,TICKMAchange_60,PCCchange_5,PCCchange_15,PCCchange_30,PCCchange_60,VOLDchange_5,VOLDchange_15,VOLDchange_30,VOLDchange_60,change_5,change_15,change_30,change_60,change_120
0,22.0,18.0,37800.0,457.47,457.98,459.20,456.75,34.27,0.8228,-0.7293,0.0,-1.940,-0.8293,-0.3427,-2.28,-1.2400,-8.02,-3.97,2.0,1.0,457.394,-937.594,-290.333,-1137.0,-2129.0,1.049,0.38,0.60,22.23,22.52,21.94,-162359662.0,-0.18,-0.04,-0.08,-1.61,-2.27,2.45,-1.73,-0.72,0.03,-4.71,-18.90,-0.93,0.0067,-0.0621,-0.1064,-0.2472,-1.460,-0.880,-0.770,-1.270,-0.0146,0.3807,1.7907,-0.0694,0.0438,0.3707,0.4907,-0.0579,0.0545,0.3934,0.6973,-0.11,-0.04,0.19,0.02,-0.0900,-0.0100,0.2500,0.1800,-0.18,-0.23,-0.19,-0.76,-0.13,-0.09,0.10,-0.17,0.04,-0.08,-0.08,0.28,0.36,0.58,-0.29,-0.29,0.00,-0.01,0.00,-0.02,-486.0,-677.0,-657.0,-543.0,-1594.394,199.406,-846.667,-104.777,-133.222,-175.777,55.334,0.009,-0.013,-0.005,0.046,-17748834.0,-15060141.0,-35012629.0,-70143145.0,-0.73,0.37,0.60,-0.63,-1.41
1,22.0,18.0,38100.0,456.74,457.91,459.22,456.59,31.01,0.8483,-0.7137,0.0,-1.370,-0.8371,-0.3448,-2.32,-1.2600,-8.12,-4.02,2.0,1.0,396.857,-872.757,-272.333,-663.0,-2255.0,1.042,0.39,0.60,22.47,22.51,21.94,-200003768.0,-0.73,-1.12,-0.92,-1.56,-1.90,2.63,-2.48,-0.15,-3.26,-1.85,-18.77,-15.29,0.0255,-0.0005,-0.0909,-0.1917,0.570,-0.580,0.170,-1.230,0.0156,0.1468,1.1063,-0.0078,-0.0113,0.3729,0.5029,-0.0021,0.0021,0.3833,0.6752,-0.04,-0.10,0.22,0.06,-0.0200,-0.0600,0.2800,0.2200,-0.10,-0.29,-0.16,-0.71,-0.05,-0.14,0.13,-0.13,0.24,0.29,0.27,0.37,0.45,0.57,-0.04,-0.53,0.01,-0.01,0.01,0.00,474.0,-429.0,191.0,242.0,-1059.857,-209.757,-390.667,18.000,-195.777,-98.333,39.778,-0.007,-0.013,-0.012,-0.029,-37644106.0,-59459525.0,-57525961.0,-98774164.0,1.10,1.48,0.34,0.21,-0.51
2,22.0,18.0,38400.0,457.84,457.95,459.15,456.75,38.07,0.8256,-0.3979,0.0,0.090,-0.5112,-0.0652,-1.97,-0.9219,-7.81,-3.67,2.0,1.0,487.667,-808.667,-160.000,503.0,-2145.0,1.036,0.39,0.60,22.13,22.48,21.94,-190070308.0,1.10,0.19,0.64,0.00,-0.32,2.40,-1.31,-1.09,7.06,3.83,-0.74,-14.00,-0.0227,0.0095,-0.0858,-0.2044,1.460,0.570,1.740,0.240,0.3158,0.3168,1.1421,0.3259,0.2487,0.6388,0.8088,0.2796,0.2196,0.5848,0.8962,0.35,0.20,0.54,0.45,0.3381,0.2281,0.5781,0.5781,0.31,0.03,0.19,-0.30,0.35,0.17,0.47,0.28,-0.34,-0.06,-0.29,-0.12,-0.02,0.54,-0.35,-0.19,0.00,0.01,0.01,0.02,1166.0,1154.0,1252.0,1156.0,15.333,-1311.667,663.000,112.333,25.556,34.333,151.556,-0.006,-0.004,-0.031,-0.051,9933460.0,-45459480.0,-54974319.0,-64391852.0,0.00,0.34,-0.77,-1.18,-1.24
3,22.0,18.0,38700.0,457.84,457.90,458.98,456.82,45.23,0.8023,-0.1375,0.0,0.335,-0.3424,0.0729,-1.79,-0.7462,-7.68,-3.50,2.0,1.0,474.867,-814.967,-171.889,323.0,-2116.0,1.063,0.41,0.60,22.13,22.49,21.94,-180273647.0,0.00,0.37,0.33,-0.70,-1.01,2.16,-

In [36]:
import joblib
df = df.reset_index(drop=True)
df.head()

,year,num,time,price,BBBasis,BBUpper,BBLower,MFI,ATR,TTM,color,Mom,WaveASlow,WaveAFast,WaveBSlow,WaveBFast,WaveCMCAD,WaveCFast,day,month,TICKBBUpper,TICKBBLower,TICKMA,TICK,ADD,PCC,TRIN,TRIN_lastclose,VIX,VIXBBUpper,VIXBBLower,VOLD,change_5_ago,change_15_ago,change_30_ago,change_60_ago,change_120_ago,BBdiff,BBUpdiff,BBLowdiff,MFIchange_5,MFIchange_15,MFIchange_30,MFIchange_60,ATRchange_5,ATRchange_15,ATRchange_30,ATRchange_60,Momchange_5,Momchange_15,Momchange_30,Momchange_60,TTMchange_5,TTMchange_15,TTMchange_30,WaveASlowchange_5,WaveASlowchange_15,WaveASlowchange_30,WaveASlowchange_60,WaveAFastchange_5,WaveAFastchange_15,WaveAFastchange_30,WaveAFastchange_60,WaveBSlowchange_5,WaveBSlowchange_15,WaveBSlowchange_30,WaveBSlowchange_60,WaveBFastchange_5,WaveBFastchange_15,WaveBFastchange_30,WaveBFastchange_60,WaveCMCADchange_5,WaveCMCADchange_15,WaveCMCADchange_30,WaveCMCADchange_60,WaveCFastchange_5,WaveCFastchange_15,WaveCFastchange_30,WaveCFastchange_60,VIXchange_5,VIXchange_15,VIXchange_30,VIXchange_60,VIXchange_120,VIXBBdiff,VIXBBUpdiff,VIXBBLowdiff,TRINchange_5,TRINchange_15,TRINchange_30,TRINchange_60,TICKchange_5,TICKchange_15,TICKchange_30,TICKchange_60,TICKBBUpdiff,TICKBBLowdiff,TICKMADiff,TICKMAchange_5,TICKMAchange_15,TICKMAchange_30,TICKMAchange_60,PCCchange_5,PCCchange_15,PCCchange_30,PCCchange_60,VOLDchange_5,VOLDchange_15,VOLDchange_30,VOLDchange_60,change_5,change_15,change_30,change_60,change_120
0,22.0,18.0,37800.0,457.47,457.98,459.20,456.75,34.27,0.8228,-0.7293,0.0,-1.940,-0.8293,-0.3427,-2.28,-1.2400,-8.02,-3.97,2.0,1.0,457.394,-937.594,-290.333,-1137.0,-2129.0,1.049,0.38,0.6,22.23,22.52,21.94,-162359662.0,-0.18,-0.04,-0.08,-1.61,-2.27,2.45,-1.73,-0.72,0.03,-4.71,-18.90,-0.93,0.0067,-0.0621,-0.1064,-0.2472,-1.460,-0.880,-0.770,-1.270,-0.0146,0.3807,1.7907,-0.0694,0.0438,0.3707,0.4907,-0.0579,0.0545,0.3934,0.6973,-0.11,-0.04,0.19,0.02,-0.0900,-0.0100,0.2500,0.1800,-0.18,-0.23,-0.19,-0.76,-0.13,-0.09,0.10,-0.17,0.04,-0.08,-0.08,0.28,0.36,0.58,-0.29,-0.29,0.00,-0.01,0.00,-0.02,-486.0,-677.0,-657.0,-543.0,-1594.394,199.406,-846.667,-104.777,-133.222,-175.777,55.334,0.009,-0.013,-0.005,0.046,-17748834.0,-15060141.0,-35012629.0,-70143145.0,-0.73,0.37,0.60,-0.63,-1.41
1,22.0,18.0,38100.0,456.74,457.91,459.22,456.59,31.01,0.8483,-0.7137,0.0,-1.370,-0.8371,-0.3448,-2.32,-1.2600,-8.12,-4.02,2.0,1.0,396.857,-872.757,-272.333,-663.0,-2255.0,1.042,0.39,0.6,22.47,22.51,21.94,-200003768.0,-0.73,-1.12,-0.92,-1.56,-1.90,2.63,-2.48,-0.15,-3.26,-1.85,-18.77,-15.29,0.0255,-0.0005,-0.0909,-0.1917,0.570,-0.580,0.170,-1.230,0.0156,0.1468,1.1063,-0.0078,-0.0113,0.3729,0.5029,-0.0021,0.0021,0.3833,0.6752,-0.04,-0.10,0.22,0.06,-0.0200,-0.0600,0.2800,0.2200,-0.10,-0.29,-0.16,-0.71,-0.05,-0.14,0.13,-0.13,0.24,0.29,0.27,0.37,0.45,0.57,-0.04,-0.53,0.01,-0.01,0.01,0.00,474.0,-429.0,191.0,242.0,-1059.857,-209.757,-390.667,18.000,-195.777,-98.333,39.778,-0.007,-0.013,-0.012,-0.029,-37644106.0,-59459525.0,-57525961.0,-98774164.0,1.10,1.48,0.34,0.21,-0.51
2,22.0,18.0,38400.0,457.84,457.95,459.15,456.75,38.07,0.8256,-0.3979,0.0,0.090,-0.5112,-0.0652,-1.97,-0.9219,-7.81,-3.67,2.0,1.0,487.667,-808.667,-160.000,503.0,-2145.0,1.036,0.39,0.6,22.13,22.48,21.94,-190070308.0,1.10,0.19,0.64,0.00,-0.32,2.40,-1.31,-1.09,7.06,3.83,-0.74,-14.00,-0.0227,0.0095,-0.0858,-0.2044,1.460,0.570,1.740,0.240,0.3158,0.3168,1.1421,0.3259,0.2487,0.6388,0.8088,0.2796,0.2196,0.5848,0.8962,0.35,0.20,0.54,0.45,0.3381,0.2281,0.5781,0.5781,0.31,0.03,0.19,-0.30,0.35,0.17,0.47,0.28,-0.34,-0.06,-0.29,-0.12,-0.02,0.54,-0.35,-0.19,0.00,0.01,0.01,0.02,1166.0,1154.0,1252.0,1156.0,15.333,-1311.667,663.000,112.333,25.556,34.333,151.556,-0.006,-0.004,-0.031,-0.051,9933460.0,-45459480.0,-54974319.0,-64391852.0,0.00,0.34,-0.77,-1.18,-1.24
3,22.0,18.0,38700.0,457.84,457.90,458.98,456.82,45.23,0.8023,-0.1375,0.0,0.335,-0.3424,0.0729,-1.79,-0.7462,-7.68,-3.50,2.0,1.0,474.867,-814.967,-171.889,323.0,-2116.0,1.063,0.41,0.6,22.13,22.49,21.94,-180273647.0,0.00,0.37,0.33,-0.70,-1.01,2.16,-1.14

In [37]:
df.isna().any().any()

False

In [38]:
X = df[['time', 'MFI', 'ATR', 'TTM', 'color', 'Mom', 'WaveASlow', 'WaveAFast', 'WaveBSlow', 'WaveBFast',
        'WaveCMCAD', 'WaveCFast', 'day', 'TICKBBUpper', 'TICKBBLower', 'TICKMA', 'TICK', 'ADD', 'PCC',
        'TRIN', 'TRIN_lastclose', 'VIX', 'VOLD', 'change_5_ago', 'change_15_ago', 'change_30_ago', 'change_60_ago',
        'change_120_ago', 'BBdiff', 'BBUpdiff', 'BBLowdiff', 'MFIchange_5', 'MFIchange_15', 'MFIchange_30', 'MFIchange_60',
        'ATRchange_5', 'ATRchange_15', 'ATRchange_30', 'ATRchange_60', 'Momchange_5', 'Momchange_15', 'Momchange_30', 
        'Momchange_60', 'TTMchange_5', 'TTMchange_15', 'TTMchange_30', 'WaveASlowchange_5', 'WaveASlowchange_15',
        'WaveASlowchange_30', 'WaveASlowchange_60', 'WaveAFastchange_5', 'WaveAFastchange_15', 'WaveAFastchange_30', 
        'WaveAFastchange_60', 'WaveBSlowchange_5', 'WaveBSlowchange_15', 'WaveBSlowchange_30', 'WaveBSlowchange_60',
        'WaveBFastchange_5', 'WaveBFastchange_15', 'WaveBFastchange_30', 'WaveBFastchange_60', 'WaveCMCADchange_5',
        'WaveCMCADchange_15', 'WaveCMCADchange_30', 'WaveCMCADchange_60', 'WaveCFastchange_5', 'WaveCFastchange_15',
        'WaveCFastchange_30', 'WaveCFastchange_60', 'VIXchange_5', 'VIXchange_15', 'VIXchange_30', 'VIXchange_60',
        'VIXchange_120', 'VIXBBdiff', 'VIXBBUpdiff', 'VIXBBLowdiff', 'TRINchange_5','TRINchange_15', 'TRINchange_30', 
        'TRINchange_60', 'TICKchange_5', 'TICKchange_15', 'TICKchange_30', 'TICKchange_60', 'TICKBBUpdiff', 'TICKBBLowdiff',
        'TICKMAchange_5', 'TICKMAchange_15', 'TICKMAchange_30', 'TICKMAchange_60', 'TICKMADiff', 'PCCchange_5', 'PCCchange_15', 
        'PCCchange_30', 'PCCchange_60', 'VOLDchange_5','VOLDchange_15', 'VOLDchange_30', 
        'VOLDchange_60']]

y = df['change_5']
        
change_pipe = make_pipeline(
    StandardScaler(),
    Ridge(alpha=0.5)
)

change_cv = pd.DataFrame(
    cross_validate(
        change_pipe,
        X,
        y,
        scoring='neg_root_mean_squared_error',
        return_train_score=True,
    )
)

change_cv

,fit_time,score_time,test_score,train_score
0,0.385586,0.015765,-0.526127,-0.364735
1,0.189251,0.016508,-0.443207,-0.390387
2,0.155960,0.016204,-0.309950,-0.420444
3,0.165368,0.016693,-0.338095,-0.420496
4,0.182895,0.018386,-0.414436,-0.399785


In [39]:
change_5_model = change_pipe.fit(X, y)
joblib.dump(change_5_model, 'SPY5Models/change_5_model_8-24.joblib')

['SPY5Models/change_5_model_8-24.joblib']

In [82]:
y = df['change_15']

change_pipe = make_pipeline(
    StandardScaler(),
    LinearRegression(),
)

change_cv = pd.DataFrame(
    cross_validate(
        change_pipe,
        X,
        y,
        scoring='neg_root_mean_squared_error',
        return_train_score=True,
    )
)

change_cv

,fit_time,score_time,test_score,train_score
0,0.563985,0.018313,-1.129121,-0.718714
1,0.508002,0.020878,-0.933779,-0.783638
2,0.508940,0.022710,-0.760967,-0.828362
3,0.360631,0.022084,-0.538224,-0.870586
4,0.485921,0.022425,-0.616814,-0.857911


In [83]:
change_15_model = change_pipe.fit(X, y)
joblib.dump(change_15_model, 'change_15_model2.joblib')

['change_15_model2.joblib']

In [84]:
y = df['change_30']

change_pipe = make_pipeline(
    StandardScaler(),
    LinearRegression(),
)

change_cv = pd.DataFrame(
    cross_validate(
        change_pipe,
        X,
        y,
        scoring='neg_root_mean_squared_error',
        return_train_score=True,
    )
)

change_cv

,fit_time,score_time,test_score,train_score
0,0.502896,0.020621,-1.693666,-1.063750
1,0.503351,0.021842,-1.402820,-1.162360
2,0.423263,0.020899,-1.118657,-1.236491
3,0.437186,0.019000,-0.796151,-1.295167
4,0.366353,0.021108,-0.907695,-1.279299


In [85]:
change_30_model = change_pipe.fit(X, y)
joblib.dump(change_30_model, 'change_30_model2.joblib')

['change_30_model2.joblib']

In [86]:
y = df['change_60']

change_pipe = make_pipeline(
    StandardScaler(),
    LinearRegression(),
)

change_cv = pd.DataFrame(
    cross_validate(
        change_pipe,
        X,
        y,
        scoring='neg_root_mean_squared_error',
        return_train_score=True,
    )
)

change_cv

,fit_time,score_time,test_score,train_score
0,0.502626,0.020886,-2.481878,-1.551743
1,0.332721,0.020797,-2.059813,-1.690618
2,0.444553,0.019396,-1.607145,-1.806985
3,0.429018,0.021129,-1.163848,-1.888589
4,0.415537,0.021546,-1.324749,-1.867874


In [87]:
change_60_model = change_pipe.fit(X, y)
joblib.dump(change_60_model, 'change_60_model2.joblib')

['change_60_model2.joblib']

In [88]:
y = df['change_120']

change_pipe = make_pipeline(
    StandardScaler(),
    LinearRegression(),
)

change_cv = pd.DataFrame(
    cross_validate(
        change_pipe,
        X,
        y,
        scoring='neg_root_mean_squared_error',
        return_train_score=True,
    )
)

change_cv

,fit_time,score_time,test_score,train_score
0,0.494393,0.015491,-3.573483,-2.247736
1,0.397970,0.020008,-2.973768,-2.429820
2,0.434661,0.020411,-2.336548,-2.594718
3,0.442337,0.020474,-1.704018,-2.711043
4,0.403481,0.019991,-1.925781,-2.682453


In [89]:
change_60_model = change_pipe.fit(X, y)
joblib.dump(change_60_model, 'change_120_model2.joblib')

['change_120_model2.joblib']

In [135]:
df

,year,num,time,price,BBBasis,BBUpper,BBLower,MFI,ATR,TTM,color,Mom,WaveASlow,WaveAFast,WaveBSlow,WaveBFast,WaveCMCAD,WaveCFast,day,month,TICKBBUpper,TICKBBLower,TICKMA,TICK,ADD,PCC,TRIN,TRIN_lastclose,VIX,VIXBBUpper,VIXBBLower,VOLD,change_5_ago,change_15_ago,change_30_ago,change_60_ago,change_120_ago,BBdiff,BBUpdiff,BBLowdiff,MFIchange_5,MFIchange_15,MFIchange_30,MFIchange_60,ATRchange_5,ATRchange_15,ATRchange_30,ATRchange_60,Momchange_5,Momchange_15,Momchange_30,Momchange_60,TTMchange_5,TTMchange_15,TTMchange_30,WaveASlowchange_5,WaveASlowchange_15,WaveASlowchange_30,WaveASlowchange_60,WaveAFastchange_5,WaveAFastchange_15,WaveAFastchange_30,WaveAFastchange_60,WaveBSlowchange_5,WaveBSlowchange_15,WaveBSlowchange_30,WaveBSlowchange_60,WaveBFastchange_5,WaveBFastchange_15,WaveBFastchange_30,WaveBFastchange_60,WaveCMCADchange_5,WaveCMCADchange_15,WaveCMCADchange_30,WaveCMCADchange_60,WaveCFastchange_5,WaveCFastchange_15,WaveCFastchange_30,WaveCFastchange_60,VIXchange_5,VIXchange_15,VIXchange_30,VIXchange_60,VIXchange_120,VIXBBdiff,VIXBBUpdiff,VIXBBLowdiff,TRINchange_5,TRINchange_15,TRINchange_30,TRINchange_60,TICKchange_5,TICKchange_15,TICKchange_30,TICKchange_60,TICKBBUpdiff,TICKBBLowdiff,TICKMADiff,TICKMAchange_5,TICKMAchange_15,TICKMAchange_30,TICKMAchange_60,PCCchange_5,PCCchange_15,PCCchange_30,PCCchange_60,VOLDchange_5,VOLDchange_15,VOLDchange_30,VOLDchange_60,change_5,change_15,change_30,change_60,change_120
0,22.0,18.0,37800,457.47,457.98,459.20,456.75,34.27,0.8228,-0.7293,0.0,-1.940,-0.8293,-0.3427,-2.2800,-1.2400,-8.0200,-3.9700,2.0,1,457.394,-937.594,-290.333,-1137.0,-2129.0,1.049,0.38,0.60,22.23,22.52,21.94,-162359662.0,-0.18,-0.04,-0.08,-1.61,-2.27,2.45,-1.73,-0.72,0.03,-4.71,-18.90,-0.93,0.0067,-0.0621,-0.1064,-0.2472,-1.460,-0.880,-0.770,-1.270,-0.0146,0.3807,1.7907,-0.0694,0.0438,0.3707,0.4907,-0.0579,0.0545,0.3934,0.6973,-0.1100,-0.0400,0.1900,0.0200,-0.0900,-0.0100,0.2500,0.1800,-0.1800,-0.2300,-0.1900,-0.7600,-0.1300,-0.0900,0.1000,-0.1700,0.04,-0.08,-0.08,0.28,0.36,0.58,-0.29,-0.29,0.00,-0.01,0.00,-0.02,-486.0,-677.0,-657.0,-543.0,-1594.394,199.406,-846.667,-104.777,-133.222,-175.777,55.334,0.009,-0.013,-0.005,0.046,-17748834.0,-15060141.0,-35012629.0,-70143145.0,-0.73,0.37,0.60,-0.63,-1.41
1,22.0,18.0,38100,456.74,457.91,459.22,456.59,31.01,0.8483,-0.7137,0.0,-1.370,-0.8371,-0.3448,-2.3200,-1.2600,-8.1200,-4.0200,2.0,1,396.857,-872.757,-272.333,-663.0,-2255.0,1.042,0.39,0.60,22.47,22.51,21.94,-200003768.0,-0.73,-1.12,-0.92,-1.56,-1.90,2.63,-2.48,-0.15,-3.26,-1.85,-18.77,-15.29,0.0255,-0.0005,-0.0909,-0.1917,0.570,-0.580,0.170,-1.230,0.0156,0.1468,1.1063,-0.0078,-0.0113,0.3729,0.5029,-0.0021,0.0021,0.3833,0.6752,-0.0400,-0.1000,0.2200,0.0600,-0.0200,-0.0600,0.2800,0.2200,-0.1000,-0.2900,-0.1600,-0.7100,-0.0500,-0.1400,0.1300,-0.1300,0.24,0.29,0.27,0.37,0.45,0.57,-0.04,-0.53,0.01,-0.01,0.01,0.00,474.0,-429.0,191.0,242.0,-1059.857,-209.757,-390.667,18.000,-195.777,-98.333,39.778,-0.007,-0.013,-0.012,-0.029,-37644106.0,-59459525.0,-57525961.0,-98774164.0,1.10,1.48,0.34,0.21,-0.51
2,22.0,18.0,38400,457.84,457.95,459.15,456.75,38.07,0.8256,-0.3979,0.0,0.090,-0.5112,-0.0652,-1.9700,-0.9219,-7.8100,-3.6700,2.0,1,487.667,-808.667,-160.000,503.0,-2145.0,1.036,0.39,0.60,22.13,22.48,21.94,-190070308.0,1.10,0.19,0.64,0.00,-0.32,2.40,-1.31,-1.09,7.06,3.83,-0.74,-14.00,-0.0227,0.0095,-0.0858,-0.2044,1.460,0.570,1.740,0.240,0.3158,0.3168,1.1421,0.3259,0.2487,0.6388,0.8088,0.2796,0.2196,0.5848,0.8962,0.3500,0.2000,0.5400,0.4500,0.3381,0.2281,0.5781,0.5781,0.3100,0.0300,0.1900,-0.3000,0.3500,0.1700,0.4700,0.2800,-0.34,-0.06,-0.29,-0.12,-0.02,0.54,-0.35,-0.19,0.00,0.01,0.01,0.02,1166.0,1154.0,1252.0,1156.0,15.333,-1311.667,663.000,112.333,25.556,34.333,151.556,-0.006,-0.004,-0.031,-0.051,9933460.0,-45459480.0,-54974319.0,-64391852.0,0.00,0.34,-0.77,-1.18,-1.24
3,22.0,18.0,38700,457.84,457.90,458.98,456.82,45.23,0.8023,-0.1375,0.0,0.335,-0.3424,0.0729,-1.7900,-0.7462,-7.6800,-3.5000,2.0,1,474.867,-814.967,-171.889,323.0,-2116.0,

In [136]:
df[(df['num'] < 12) | (df['year'] < 24) | (df['month'] <= 3)]
df

,year,num,time,price,BBBasis,BBUpper,BBLower,MFI,ATR,TTM,color,Mom,WaveASlow,WaveAFast,WaveBSlow,WaveBFast,WaveCMCAD,WaveCFast,day,month,TICKBBUpper,TICKBBLower,TICKMA,TICK,ADD,PCC,TRIN,TRIN_lastclose,VIX,VIXBBUpper,VIXBBLower,VOLD,change_5_ago,change_15_ago,change_30_ago,change_60_ago,change_120_ago,BBdiff,BBUpdiff,BBLowdiff,MFIchange_5,MFIchange_15,MFIchange_30,MFIchange_60,ATRchange_5,ATRchange_15,ATRchange_30,ATRchange_60,Momchange_5,Momchange_15,Momchange_30,Momchange_60,TTMchange_5,TTMchange_15,TTMchange_30,WaveASlowchange_5,WaveASlowchange_15,WaveASlowchange_30,WaveASlowchange_60,WaveAFastchange_5,WaveAFastchange_15,WaveAFastchange_30,WaveAFastchange_60,WaveBSlowchange_5,WaveBSlowchange_15,WaveBSlowchange_30,WaveBSlowchange_60,WaveBFastchange_5,WaveBFastchange_15,WaveBFastchange_30,WaveBFastchange_60,WaveCMCADchange_5,WaveCMCADchange_15,WaveCMCADchange_30,WaveCMCADchange_60,WaveCFastchange_5,WaveCFastchange_15,WaveCFastchange_30,WaveCFastchange_60,VIXchange_5,VIXchange_15,VIXchange_30,VIXchange_60,VIXchange_120,VIXBBdiff,VIXBBUpdiff,VIXBBLowdiff,TRINchange_5,TRINchange_15,TRINchange_30,TRINchange_60,TICKchange_5,TICKchange_15,TICKchange_30,TICKchange_60,TICKBBUpdiff,TICKBBLowdiff,TICKMADiff,TICKMAchange_5,TICKMAchange_15,TICKMAchange_30,TICKMAchange_60,PCCchange_5,PCCchange_15,PCCchange_30,PCCchange_60,VOLDchange_5,VOLDchange_15,VOLDchange_30,VOLDchange_60,change_5,change_15,change_30,change_60,change_120
0,22.0,18.0,37800,457.47,457.98,459.20,456.75,34.27,0.8228,-0.7293,0.0,-1.940,-0.8293,-0.3427,-2.2800,-1.2400,-8.0200,-3.9700,2.0,1,457.394,-937.594,-290.333,-1137.0,-2129.0,1.049,0.38,0.60,22.23,22.52,21.94,-162359662.0,-0.18,-0.04,-0.08,-1.61,-2.27,2.45,-1.73,-0.72,0.03,-4.71,-18.90,-0.93,0.0067,-0.0621,-0.1064,-0.2472,-1.460,-0.880,-0.770,-1.270,-0.0146,0.3807,1.7907,-0.0694,0.0438,0.3707,0.4907,-0.0579,0.0545,0.3934,0.6973,-0.1100,-0.0400,0.1900,0.0200,-0.0900,-0.0100,0.2500,0.1800,-0.1800,-0.2300,-0.1900,-0.7600,-0.1300,-0.0900,0.1000,-0.1700,0.04,-0.08,-0.08,0.28,0.36,0.58,-0.29,-0.29,0.00,-0.01,0.00,-0.02,-486.0,-677.0,-657.0,-543.0,-1594.394,199.406,-846.667,-104.777,-133.222,-175.777,55.334,0.009,-0.013,-0.005,0.046,-17748834.0,-15060141.0,-35012629.0,-70143145.0,-0.73,0.37,0.60,-0.63,-1.41
1,22.0,18.0,38100,456.74,457.91,459.22,456.59,31.01,0.8483,-0.7137,0.0,-1.370,-0.8371,-0.3448,-2.3200,-1.2600,-8.1200,-4.0200,2.0,1,396.857,-872.757,-272.333,-663.0,-2255.0,1.042,0.39,0.60,22.47,22.51,21.94,-200003768.0,-0.73,-1.12,-0.92,-1.56,-1.90,2.63,-2.48,-0.15,-3.26,-1.85,-18.77,-15.29,0.0255,-0.0005,-0.0909,-0.1917,0.570,-0.580,0.170,-1.230,0.0156,0.1468,1.1063,-0.0078,-0.0113,0.3729,0.5029,-0.0021,0.0021,0.3833,0.6752,-0.0400,-0.1000,0.2200,0.0600,-0.0200,-0.0600,0.2800,0.2200,-0.1000,-0.2900,-0.1600,-0.7100,-0.0500,-0.1400,0.1300,-0.1300,0.24,0.29,0.27,0.37,0.45,0.57,-0.04,-0.53,0.01,-0.01,0.01,0.00,474.0,-429.0,191.0,242.0,-1059.857,-209.757,-390.667,18.000,-195.777,-98.333,39.778,-0.007,-0.013,-0.012,-0.029,-37644106.0,-59459525.0,-57525961.0,-98774164.0,1.10,1.48,0.34,0.21,-0.51
2,22.0,18.0,38400,457.84,457.95,459.15,456.75,38.07,0.8256,-0.3979,0.0,0.090,-0.5112,-0.0652,-1.9700,-0.9219,-7.8100,-3.6700,2.0,1,487.667,-808.667,-160.000,503.0,-2145.0,1.036,0.39,0.60,22.13,22.48,21.94,-190070308.0,1.10,0.19,0.64,0.00,-0.32,2.40,-1.31,-1.09,7.06,3.83,-0.74,-14.00,-0.0227,0.0095,-0.0858,-0.2044,1.460,0.570,1.740,0.240,0.3158,0.3168,1.1421,0.3259,0.2487,0.6388,0.8088,0.2796,0.2196,0.5848,0.8962,0.3500,0.2000,0.5400,0.4500,0.3381,0.2281,0.5781,0.5781,0.3100,0.0300,0.1900,-0.3000,0.3500,0.1700,0.4700,0.2800,-0.34,-0.06,-0.29,-0.12,-0.02,0.54,-0.35,-0.19,0.00,0.01,0.01,0.02,1166.0,1154.0,1252.0,1156.0,15.333,-1311.667,663.000,112.333,25.556,34.333,151.556,-0.006,-0.004,-0.031,-0.051,9933460.0,-45459480.0,-54974319.0,-64391852.0,0.00,0.34,-0.77,-1.18,-1.24
3,22.0,18.0,38700,457.84,457.90,458.98,456.82,45.23,0.8023,-0.1375,0.0,0.335,-0.3424,0.0729,-1.7900,-0.7462,-7.6800,-3.5000,2.0,1,474.867,-814.967,-171.889,323.0,-2116.0,